# Mario PPO RL Project

I'll start by saying this was my very first reinforcement learning project I've even attempted. I didn't implement PPO myself from scratch, but I learned a whole lot about what goes into such projects - preprocessing, understanding the environment (an NES emulator in this case), and a lifetime of waiting on my computer!

I watched about a dozen different YouTube videos on AIs that play various games and found the whole concept incredibly intriguing watching how so many algorithms were used. I was eager to learn the differences and to know when one algorithm might be considered a better choice than another. I love tackling challenges that would bring me to new heights in my programming knowledge, so Mario came to my mind as the perfect fit for my new expedition into machine learning.

### Fun Facts

1. The whole NES system runs on 2 Kb of RAM and a 1.79 MHz processor! The game screen is 240 wide by 256 pixels tall.
2. Due to how early power systems were developed, the North American (NTSC) version of the game runs at 60 fps, and there are two European (PAL) versions - one version is optimized for PAL and the other is not. The unoptimized PAL version runs slower and has slower music, while the optimized version keeps pace with the NTSC version, but occasionally has to skip a frame to do so. This leads to frame-perfect or pixel-perfect tricks being slightly different.
3. The NES has 5 channels for sound - 3 were usable for the game's music and audio effects. This is why some of the music stopped if you collected a coin or powerup. This also forced the composers of the era to get creative with the limited number of parts their song could have.
4. Licking your old NES cartridges to get them to work is just a placebo effect and you are actually making the problem worse by doing so!
5. The average person has a reaction time of about 273ms (or 16 frames in Mario terms) and can mash a controller button 6.69 times every second (or once every 9 frames). My initial AI was not limited by these factor.

## Starting the Game

### Installing the dependencies

There were several things for me to install for this project. `gym`, `gym_super_mario_bros`, and `nes_py` were necessary to get the game running. The game itself is the NTSC version running on an emulator. `nes_py` makes hacking the ram and cheating with the game quite easy! This is handy if I want to make any sort of save states to revert to.

I am using a custom `PyTorch` installation which will let me use my GPU (a little GeForce GTX 1050 Ti) via CUDA and `stable-baselines3` for the actual PPO model. I didn't do any multiprocessing in this project, which would be a next step for me to learn to drastically increase my performance gains.

In [1]:
# Install the environment
!pip install gym==0.21.0 gym_super_mario_bros==7.3.0 nes_py --no-input

# This line must be run before the installation of stable-baselines3 to force
# PyTorch to use the GPU via CUDA.
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117 --no-input

# Stable-baselines3 gives us access to many RL models.
!pip install stable-baselines3[extra] --no-input

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
Defaulting to user installation because normal site-packages is not writeable


### Understanding the Game

The NES controller has a total of 8 buttons - `up`, `down`, `left`, `right`, `A`, `B`, `start`, and `select`. On the emulator, all of these buttons could be pressed at the same time, even if such configurations are physically impossible such as `up` and `down` simultaneously. This gives a total of 256 possible input combinations per frame.

We need to limit the action space so the agent can learn easier. We don't want the agent pushing start or select for any reason. Invalid combinations could be avoided as well. Lastly, certain combinations don't yield any special results, such as pushing `down` in conjunction with any other button.

The goal is to run to the right as quick as possible, and the environment rewards Mario based on his change in X position. (Rewarding Mario for the actual score on the screen will cause him to mope around collecting all the coins and grinding enemies.) I decided to force the AI to hold `right` and `B` to constantly run right. All it has to decide is when to push the `A` button to jump. I figured this would net the quickest results as there are no parts of the game where you are forced to move left or stop moving.

The environment will only return frames which are rewardable, so the start screen is skipped and 1 player mode is automatically selected. Similarly, death and other animations are also skipped.

In [2]:
# Import the game and SIMPLIFIED controls
import gym_super_mario_bros
from gym_super_mario_bros.actions import *
SPEEDRUN_MOVEMENT = [['right', 'B'], ['right', 'A', 'B']]

# Import wrappers
from gym import Wrapper
from nes_py.wrappers import JoypadSpace
from gym.wrappers import GrayScaleObservation, RecordEpisodeStatistics, ResizeObservation
from stable_baselines3.common.vec_env import VecFrameStack, SubprocVecEnv

# Import the RL model
from stable_baselines3 import PPO

# Import utilities for image viewing, file I/O, RNG, etc.
from stable_baselines3.common.callbacks import BaseCallback
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

C:\Users\Blake\AppData\Roaming\Python\Python39\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding environment ALE/Tetris-v5
  logger.warn("Overriding environment {}".format(id))
C:\Users\Blake\AppData\Roaming\Python\Python39\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding environment ALE/Tetris-ram-v5
  logger.warn("Overriding environment {}".format(id))
C:\Users\Blake\AppData\Roaming\Python\Python39\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding environment ALE/Adventure-v5
  logger.warn("Overriding environment {}".format(id))
C:\Users\Blake\AppData\Roaming\Python\Python39\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding environment ALE/Adventure-ram-v5
  logger.warn("Overriding environment {}".format(id))
C:\Users\Blake\AppData\Roaming\Python\Python39\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding environment ALE/AirRaid-v5
  logger.warn("Overriding environm

## Testing the Environment

The specific environment I used was one which selected a random stage on each reset. I did this to help the AI generalize as well as to reduce overfitting on the first level. Plus, it would be very slow if Mario had to go through all 7 of the other worlds before getting to world 8 which the AI needs to learn.

This had me thinking of implementing a similar idea for a single level as well - what if we could rewind a second or two and the AI could retry just the part where it died instead of having to run through the first half of the level? There were some pitfalls in this logic about perhaps getting stuck in death loops, so I did not implement this until later to measure the improvements that could be made.

The environment also turns everything into rectangles to reduce the complexity of the information for the agent. The code below tests the game for a few seconds to ensure that it is working properly.

In [3]:
# Create and wrap the environment
env = gym_super_mario_bros.make('SuperMarioBros-v3')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
state = env.reset()
env.render()

# Test the game with random actions
try:
    for step in range(10 ** 3):
        # Pick a random action
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        
        if done:
            env.reset()
        env.render()
except KeyboardInterrupt:
    pass
finally:
    env.close()

C:\Users\Blake\AppData\Roaming\Python\Python39\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))
C:\Users\Blake\AppData\Roaming\Python\Python39\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


## Preprocessing

The design choices I made here were intended to help speed up the rate that the AI learns and to model human reactions a little better. The base environment that I used was a rectangular downsample of the whole game - sprites are reduced to only their hitboxes. I also grayscale the image which could possibly cut out on important information (such as not being able to distinguish red lava and a gray floor) but will overall speed up the learning process.

Original | Downsampled | Grayscaled
- | - | -
![mario original](smb_original.png) | ![mario downsampled](smb_rectangle.png) | ![mario grayscale](smb_grayscale.png)

### Deadlock

I decided if Mario gets stuck and does not progress for 3 seconds then he should be killed off without having to wait minutes for the timer to run out in order to restart.

In [4]:
class DeadlockEnv(Wrapper):
    
    def __init__(self, env, threshold):
        super().__init__(env)
        self.threshold = threshold
    
    def reset(self, **kwargs):
        self.max_x_pos = 0
        self.count = 0
        return self.env.reset(**kwargs)
    
    def step(self, action):
        state, reward, done, info = self.env.step(action)
        x_pos = info['x_pos']
        if x_pos <= self.max_x_pos:
            self.count += 1
        else:
            self.count = 0
            self.max_x_pos = x_pos
        
        if self.count >= self.threshold:
            reward -= 15
            done = True
        
        return state, reward, done, info

### Skipping Frames

The AI can change inputs every 5 frames to model human limitations. This also makes it easier to do things like jump over pipes, where the a button needs to be held for several frames. The reward for the frames is summated.

In [5]:
class SkipFrame(Wrapper):
    
    def __init__(self, env, skip):
        super().__init__(env)
        self._skip = skip
    
    def step(self, action):
        total_reward = 0
        for i in range(self._skip):
            state, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return state, total_reward, done, info

### Other Wrappers

I also grayscale the image and reduced the size by half to reduce the information passed to the AI. (If you try to display the image using Matplotlib, it messes with the colors on a grayscale image when rendering the output for visibility.)

The environment is put into a subprocess environment for the PPO algorithm to play multiple games at once, utilizing the GPU.

Then, a total of 4 frames are stacked on each other to give the AI a sense of motion and acceleration.

Now we just create the actual environment and wrap everything up.

In [6]:
def create_env(ranks, seed = 0):
    
    def helper(rank):
        # 1. Create the base environment
        env = gym_super_mario_bros.make('SuperMarioBros-v3')
        env.seed(seed + rank)

        # 2. Simplify the controls
        env = JoypadSpace(env, COMPLEX_MOVEMENT)

        # 3. If Mario is not making progress after 180 frames (3 seconds), end it
        env = DeadlockEnv(env, 60 * 3)

        # 4. Skip frames so the agent can only change button presses every 5 frames
        # env = SkipFrame(env, 5)

        # 5. Grayscale - This cuts down on info for the agent to learn
        env = GrayScaleObservation(env, keep_dim=True)
        
        # 6. Resize the image to 50%
        env = ResizeObservation(env, (120, 128))
        
        # 7. Record stats about the env for tensorboard
        env = RecordEpisodeStatistics(env)
        
        return env

    # 8. Wrap inside Dummy Vector Env
    env = SubprocVecEnv([(lambda: helper(i)) for i in range(ranks)])

    # 9. Stack the frames
    env = VecFrameStack(env, 4, channels_order='last')
    
    return env

## Logging and Training

Initially I decided to train the model for a total of 1 million timesteps and benchmark the progress every 10,000 steps. I used a learning rate of 1e-6 and chose to update the model every 512 steps.

In [7]:
# Helper class will save our model every n steps
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, save_freq, save_path, verbose = 1):
        super().__init__(verbose)
        self.save_freq = save_freq
        self.save_path = save_path
    
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
    
    def _on_step(self):
        if self.n_calls % self.save_freq == 0:
            model_path = os.path.join(self.save_path,
                                      f'best_model_{self.n_calls // 1000}k')
            self.model.save(model_path)
        return True

In [8]:
# Set up save dirs
CHECKPOINT_DIR = 'E:/train/'
TRAIN_DIR = 'E:/logs/'
MODEL_PATH = 'E:/train/finished_model'
REWARD_LOG = TRAIN_DIR + 'reward_log.csv'

# Set up constants
SAVE_FREQ = 10 ** 5
NUM_SUBPROCESSES = 4

# Set up the model saving callback
callback = TrainAndLoggingCallback(SAVE_FREQ // NUM_SUBPROCESSES,
                                   CHECKPOINT_DIR)
env = create_env(NUM_SUBPROCESSES)

In [9]:
# Create the model (policy)
model = PPO('CnnPolicy',
            env = env,
            verbose = 1,
            tensorboard_log = TRAIN_DIR,
            learning_rate = 1e-5,
            n_steps = 512)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [10]:
try:
    # Train and save model
    model.learn(total_timesteps = 10 ** 6,
                callback = callback)
    model.save(MODEL_PATH)
except KeyboardInterrupt:
    pass
finally:
    # Free memory, close resources
    del model
    del env
    print('Freed memory and closed resources.')

Logging to E:/logs/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 207      |
|    ep_rew_mean     | -34.4    |
| time/              |          |
|    fps             | 283      |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 302         |
|    ep_rew_mean          | 9.75        |
| time/                   |             |
|    fps                  | 236         |
|    iterations           | 2           |
|    time_elapsed         | 17          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015081482 |
|    clip_fraction        | 0.123       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.48       |
|    explained_variance   | 0.000377    |
|    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 615         |
|    ep_rew_mean          | 336.82352   |
| time/                   |             |
|    fps                  | 209         |
|    iterations           | 11          |
|    time_elapsed         | 107         |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.010392243 |
|    clip_fraction        | 0.108       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.28       |
|    explained_variance   | 0.895       |
|    learning_rate        | 1e-05       |
|    loss                 | 28.1        |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.00772    |
|    value_loss           | 105         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 610   

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 592        |
|    ep_rew_mean          | 417.9014   |
| time/                   |            |
|    fps                  | 207        |
|    iterations           | 21         |
|    time_elapsed         | 207        |
|    total_timesteps      | 43008      |
| train/                  |            |
|    approx_kl            | 0.01372692 |
|    clip_fraction        | 0.141      |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.04      |
|    explained_variance   | 0.975      |
|    learning_rate        | 1e-05      |
|    loss                 | 23.6       |
|    n_updates            | 200        |
|    policy_gradient_loss | -0.00401   |
|    value_loss           | 59.3       |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 588         |
|    ep_rew_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 608         |
|    ep_rew_mean          | 497.37      |
| time/                   |             |
|    fps                  | 206         |
|    iterations           | 31          |
|    time_elapsed         | 307         |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.007682711 |
|    clip_fraction        | 0.0864      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.66       |
|    explained_variance   | 0.913       |
|    learning_rate        | 1e-05       |
|    loss                 | 252         |
|    n_updates            | 300         |
|    policy_gradient_loss | -0.00283    |
|    value_loss           | 287         |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 590 

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 566         |
|    ep_rew_mean          | 561.08      |
| time/                   |             |
|    fps                  | 206         |
|    iterations           | 41          |
|    time_elapsed         | 407         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.007549705 |
|    clip_fraction        | 0.178       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.68       |
|    explained_variance   | 0.986       |
|    learning_rate        | 1e-05       |
|    loss                 | 32.4        |
|    n_updates            | 400         |
|    policy_gradient_loss | -0.0032     |
|    value_loss           | 60.9        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 562   

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 584         |
|    ep_rew_mean          | 749.67      |
| time/                   |             |
|    fps                  | 204         |
|    iterations           | 441         |
|    time_elapsed         | 4426        |
|    total_timesteps      | 903168      |
| train/                  |             |
|    approx_kl            | 0.008060316 |
|    clip_fraction        | 0.0717      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.608      |
|    explained_variance   | 0.894       |
|    learning_rate        | 1e-05       |
|    loss                 | 226         |
|    n_updates            | 4400        |
|    policy_gradient_loss | -0.00514    |
|    value_loss           | 316         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 591   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 592         |
|    ep_rew_mean          | 765.98      |
| time/                   |             |
|    fps                  | 204         |
|    iterations           | 451         |
|    time_elapsed         | 4526        |
|    total_timesteps      | 923648      |
| train/                  |             |
|    approx_kl            | 0.039807193 |
|    clip_fraction        | 0.148       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.777      |
|    explained_variance   | 0.963       |
|    learning_rate        | 1e-05       |
|    loss                 | 19.3        |
|    n_updates            | 4500        |
|    policy_gradient_loss | -0.0142     |
|    value_loss           | 94.2        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 595   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 610         |
|    ep_rew_mean          | 728.22      |
| time/                   |             |
|    fps                  | 204         |
|    iterations           | 461         |
|    time_elapsed         | 4624        |
|    total_timesteps      | 944128      |
| train/                  |             |
|    approx_kl            | 0.025326822 |
|    clip_fraction        | 0.145       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.85       |
|    explained_variance   | 0.989       |
|    learning_rate        | 1e-05       |
|    loss                 | 8.68        |
|    n_updates            | 4600        |
|    policy_gradient_loss | -0.0135     |
|    value_loss           | 28.1        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 625   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 647         |
|    ep_rew_mean          | 725.03      |
| time/                   |             |
|    fps                  | 204         |
|    iterations           | 471         |
|    time_elapsed         | 4722        |
|    total_timesteps      | 964608      |
| train/                  |             |
|    approx_kl            | 0.008841213 |
|    clip_fraction        | 0.119       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.942      |
|    explained_variance   | 0.991       |
|    learning_rate        | 1e-05       |
|    loss                 | 6.59        |
|    n_updates            | 4700        |
|    policy_gradient_loss | -0.00535    |
|    value_loss           | 22.7        |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 649 

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 653        |
|    ep_rew_mean          | 704.18     |
| time/                   |            |
|    fps                  | 204        |
|    iterations           | 481        |
|    time_elapsed         | 4821       |
|    total_timesteps      | 985088     |
| train/                  |            |
|    approx_kl            | 0.00989325 |
|    clip_fraction        | 0.0854     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.727     |
|    explained_variance   | 0.969      |
|    learning_rate        | 1e-05      |
|    loss                 | 16         |
|    n_updates            | 4800       |
|    policy_gradient_loss | -0.00339   |
|    value_loss           | 63.7       |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 652         |
|    ep_rew_m

## Testing the Model

In [11]:
# Load the model
model = PPO.load('E:/Train/finished_model.zip',
                 #env = create_env(4),
                 reset_num_timesteps = False,
                 tensorboard_log=TRAIN_DIR)

In [ ]:
env = create_env(1)
state = env.reset()
try:
    while True:
        action, _ = model.predict(state)
        state, reward, done, info = env.step(action)
        env.render()
except KeyboardInterrupt:
    pass
finally:
    env.close()

## The Results



# Future improvements

Some basics are lowering the learning rate and giving the model more time to run. Other algorithms and model architectures besides PPO could also be tried. If the model is running into pixel perfect boundaries, the size of the screen could be kept the same. If some color palletes clash on the grayscale image, then the full RGB bands could be used as well. However, these will slow down training.

One issue the AI faces is having to spend a lot of time playing through the first part of a level in order to get to a new part to learn. This is especially troublesome if we would all the levels to be played. The gym-super-mario-bros environment allows Mario to start on a random world which can deal with this. Another remedy is to save the current state every second (up to 5 saved states) and when Mario dies, revert back to the state from 5 seconds ago to retry the new part again.